In [10]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from __future__ import print_function

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

In [11]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

from WordVector import WordVector

In [12]:
import data_helper
_,_,relation = data_helper.get_triplet()
relation[:5]

['cause', 'cause', 'cause', 'cause', 'cause']

In [13]:
relation = list(set(relation))
relation[:5]

['reduce', 'incidence', 'inject', 'acquire', 'evoke']

In [14]:
wv = WordVector(relation)

Loading a pre-trained model...
Load success!


In [15]:
wv.word[:5]

['reduce', 'incidence', 'inject', 'acquire', 'evoke']

In [16]:
wv.vector[0]

array([-2.9233799 ,  0.27068427,  3.5848484 ,  0.61355835, -1.86760664,
       -1.5554688 , -2.91399121,  3.18267393, -0.19483082, -1.67424166,
        1.90843332,  0.03602009, -1.38194168,  1.72313547,  2.12456441,
       -1.00012076, -1.31767321, -0.54709303, -1.92937732, -0.86250794,
        2.63512778,  1.86491048, -2.6611917 ,  2.17005444,  1.93976545,
       -2.30371737, -0.0474452 , -2.13256645, -1.46955848, -2.20100093,
        9.57727814, -1.28390205,  0.35321918,  3.11763549,  0.94683415,
        0.52200979,  2.72153187,  2.60473013,  0.73925871, -1.95169747,
        2.28206921, -4.69458389,  3.99521279,  2.24136806,  1.04797208,
       -3.25955391,  3.4405241 ,  2.13889623, -4.89090204,  0.76477838,
       -2.98822904, -1.39674115, -1.99379635, -1.98837137, -1.44772589,
        0.80073178,  0.22555631, -2.19846797, -1.21672571,  0.57123613,
        2.49323416, -0.31888425,  0.77453929, -1.03983855, -0.71223044,
        3.3141408 ,  0.9258768 , -0.05844536, -1.14821315, -5.38

# PCA: 100dim -> 5dim

In [17]:
from DimensionReduction import DimensionReduction
dr = DimensionReduction()
vec_3d = dr.PCA(wv.vector, dim=5)
vec_3d[:3]

Explained variation per principal component: [ 0.11155301  0.07719801  0.06918816  0.06299675  0.04923867]
Average of Explained variations: 0.3701745867729187


array([[ -9.74731255,  -1.49740481,  -3.31040549,   5.10130739,
          4.09657431],
       [ 10.61711597,  -8.81287003,  -1.75401568,  -3.01456094,
         -6.40796185],
       [ -2.12795329,   2.04488659, -11.16493988,   3.9135778 ,
          1.48228574]], dtype=float32)

# Silhouette Coefficient Analysis

In [18]:
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
colors=["rgb(255,0,0)", "rgb(0,255,0)", "rgb(0,0,255)", "rgb(255,255,0)", "rgb(255,0,255)", 
        "rgb(0,255,255)", "rgb(127,0,255)", "rgb(255,128,0)","rgb(255,0,127)", "rgb(128,128,128)"]

In [34]:
from DimensionReduction import DimensionReduction
dr = DimensionReduction()

vec_3d = dr.PCA(wv.vector, dim=3)
vec_5d = dr.PCA(wv.vector, dim=5)

Explained variation per principal component: [ 0.11155301  0.07719801  0.06918816]
Average of Explained variations: 0.2579391598701477
Explained variation per principal component: [ 0.11155301  0.07719801  0.06918816  0.06299675  0.04923867]
Average of Explained variations: 0.3701745867729187


In [35]:
figures = []

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig = tools.make_subplots(rows=1, cols=2,
                              print_grid=False,
                              specs=[[{'is_3d': False}, {'is_3d': True}]])

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    fig['layout']['xaxis1'].update(title='The silhouette coefficient values', range=[-0.1, 1])
   
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    fig['layout']['yaxis1'].update(title='Cluster label',
                                   showticklabels=False,
                                   range=[0, len(vec_5d) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters)
    cluster_labels = clusterer.fit_predict(vec_5d)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(vec_5d, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(vec_5d, cluster_labels)
    y_lower = 10
    
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        # colors = cm.spectral(cluster_labels.astype(float) / n_clusters)

        filled_area = go.Scatter(y=np.arange(y_lower, y_upper),
                                 x=ith_cluster_silhouette_values,
                                 mode='lines',
                                 showlegend=False,
                                 line=dict(width=0.5,
                                          color=colors[i]),
                                 fill='tozerox')
        fig.append_trace(filled_area, 1, 1)
        
        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples
        

    # The vertical line for average silhouette score of all the values
    axis_line = go.Scatter(x=[silhouette_avg],
                           y=[0, len(vec_3d) + (n_clusters + 1) * 10],
                           showlegend=False,
                           mode='lines',
                           line=dict(color="red", dash='dash',
                                     width =1) )

    fig.append_trace(axis_line, 1, 1)
    
    
    
    
    
    
    # 2nd Plot showing the actual clusters formed
    
    labeled_vec = list(zip(vec_3d, cluster_labels))
    # Labeling the clusters
    centers_ = clusterer.cluster_centers_
    for i in range(n_clusters):
        ith_cluster_vector = np.array([v for (v,c) in labeled_vec if c == i])
        # colors = matplotlib.colors.colorConverter.to_rgb(cm.spectral(float(i) / n_clusters))
        # colors = 'rgb'+str(colors)
        
        clusters = go.Scatter3d(x=ith_cluster_vector[:, 0], 
                                y=ith_cluster_vector[:, 1], 
                                z=ith_cluster_vector[:, 2],
                                showlegend=False,
                                mode='markers',
                                marker=dict(
                                    color=colors[i],
                                    size=3,
                                    line=dict(
                                        width=0.5
                                    )
                                )
                               )
        fig.append_trace(clusters, 1, 2)
        
        # Draw white circles at cluster centers
        centers = go.Scatter3d(x=centers_[i][0], 
                               y=centers_[i][1],
                               z=centers_[i][2],
                               showlegend=False,
                               mode='markers',
                               marker=dict(
                                   color=colors[i], size=5,
                                   line=dict(color='black', width=1))
                               )

        fig.append_trace(centers, 1, 2)
    
                      
        
        
        

    fig['layout'].update(title="Silhouette analysis for KMeans clustering on sample data "
                         "with n_clusters = %d" % n_clusters)
    
    figures.append(fig)

For n_clusters = 2 The average silhouette_score is : 0.202604
For n_clusters = 3 The average silhouette_score is : 0.205287
For n_clusters = 4 The average silhouette_score is : 0.240217
For n_clusters = 5 The average silhouette_score is : 0.242035
For n_clusters = 6 The average silhouette_score is : 0.234816
For n_clusters = 7 The average silhouette_score is : 0.240444
For n_clusters = 8 The average silhouette_score is : 0.240617
For n_clusters = 9 The average silhouette_score is : 0.235131
For n_clusters = 10 The average silhouette_score is : 0.237353


In [31]:
py.iplot(figures[0])

In [33]:
py.iplot(figures[1])

PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per day. If you need to raise your daily limit, consider upgrading to a Student or Personal Plan (see: https://plot.ly/products/cloud).

In [ ]:
py.iplot(figures[2])